# Parte 2 - Registro de Modelos de Machine Learning


## Objetivos:

1. Entrena un modelo de regresión logística con datos de ejemplo (scikit-learn).

2. Registra hiperparámetros (C, regularización, solver) y métricas (accuracy, f1-score).

3. Usa mlflow.sklearn.log_model() para guardar el modelo.

4. Regístralo en el Model Registry con un nombre único.

5. Promuévelo de “None” → “Staging” → “Production”.

## Importe de librerías

In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from mlflow.tracking import MlflowClient
import os

## Configurar MLflow

In [2]:
os.makedirs("mlruns", exist_ok=True)
mlflow.set_tracking_uri("file://" + os.path.abspath("mlruns"))
mlflow.set_experiment("regresion_logistica_MLFlow")

2025/10/29 11:14:27 INFO mlflow.tracking.fluent: Experiment with name 'regresion_logistica_MLFlow' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///Users/anderson/Desktop/proyectos '
 'GIT/MLOps/Laboratorio-MLFLOW/mlruns/366788345699976599'), creation_time=1761754467811, experiment_id='366788345699976599', last_update_time=1761754467811, lifecycle_stage='active', name='regresion_logistica_MLFlow', tags={}>

## Carga del dataset

In [3]:
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42
)

## Entrenamiento del modelo

In [4]:
C = 1.0
solver = "lbfgs"
max_iter = 200

model = LogisticRegression(C=C, solver=solver, max_iter=max_iter)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,200
,multi_class,'deprecated'


## Evaluación de métricas

In [5]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="macro")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 1.0000
F1-score: 1.0000


## Registro en MLFlow

In [6]:
with mlflow.start_run():
    
    mlflow.log_param("C", C)
    mlflow.log_param("solver", solver)
    mlflow.log_param("max_iter", max_iter)
    
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1)
    
    
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="Iris_regression_logistica_MLFlow"
    )

2025/10/29 11:14:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/29 11:14:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Iris_regression_logistica_MLFlow'.
Created version '1' of model 'Iris_regression_logistica_MLFlow'.


## Promover el modelo en el Model Registry

In [8]:
client = MlflowClient()
model_name = "Iris_regression_logistica_MLFlow"

latest_version = client.get_latest_versions(model_name, stages=["None"])[0].version

# Promover a Staging
client.transition_model_version_stage(
    name=model_name,
    version=latest_version,
    stage="Staging"
)

# Promover a Production
client.transition_model_version_stage(
    name=model_name,
    version=latest_version,
    stage="Production"
)

# Agregar descripción
client.update_model_version(
    name=model_name,
    version=latest_version,
    description=f"Modelo de regresión logística sobre Iris usando MLFlow. Accuracy={accuracy:.4f}, F1={f1:.4f}."
)

print("Modelo promovido a 'Production'.")

Modelo promovido a 'Production'.


/var/folders/69/gxwhgd3s2v73lfrd296fzzs40000gn/T/ipykernel_70968/2579556896.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(model_name, stages=["None"])[0].version
/var/folders/69/gxwhgd3s2v73lfrd296fzzs40000gn/T/ipykernel_70968/2579556896.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
/var/folders/69/gxwhgd3s2v73lfrd29